In [1]:
from bs4 import BeautifulSoup
import os
import re
import json
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy3
morph = pymorphy3.MorphAnalyzer(lang='ru')
nltk.download('stopwords')
directory = 'YaKlass'
try:
    os.chdir(directory)
except:
    pass


importing Jupyter notebook from To_show_code.ipynb


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def read_soup(path):
    with open(path, 'r', encoding = 'utf-8') as file:
        return BeautifulSoup(file.read(), "html.parser")
    
def write(path, text):
    with open(path, 'w', encoding = 'utf-8') as file:
        file.write(str(text))
    
def text_from_soup(soup):
    '''Извлекает весь текст во всех тэгах элемента soup в строку'''
    sents_by_strip = " ".join(item.strip() for item in soup.find_all(text=True)) #находит все тэги с текстом внутри
    sents_by_strip = re.sub("\s+", " ", sents_by_strip)
    return sents_by_strip

def score(word):
    return morph.parse(word)[0].score

def merging_broken_splitted_words(words):
    '''объединяет слова, например, "Т" и "реугольник" объединятся в "Треугольник"
    работает всегда, кроме случаев, когда идут 2 и более однобуквенных слова'''
    #print('words ', words)
    if ('l' in words) or ('m' in words): # сложный случай, будет обработка вручную
        return None
    
    new_words = []
    words_was_joined = False
    for ind, word in enumerate(words):
        if ind == len(words) - 1:
            if words_was_joined == False:
                new_words.append(word)
            break
        if len(word) == 1:
            new_word = words[ind] + words[ind + 1]
            if score(new_word) > score(words[ind + 1]):
                new_words.append(new_word)
                words_was_joined = True
            else:
                new_words.append(word)
        elif words_was_joined == False:
            new_words.append(word)
        else:
            words_was_joined = False
    return new_words

def term_keyword_count(sent, params = ['—', 'называть', 'называться', 'является', 'являться']):
    acceptable_values = ['—', 'называть', 'называться', 'является', 'являться']
    for param_ in params:
        if param_ not in acceptable_values:
            print('Недопустимое keyword для извлечения терминов')
            return
    count = 0
    words = [morph.parse(word)[0].normal_form for word in word_tokenize(sent)]
    for param_ in params:
        count += words.count(param_)
    return count

In [ ]:
import sqlite3
def create_db():
   conn = sqlite3.connect('../definitions.db')
   cur = conn.cursor()
   cur.execute("""CREATE TABLE IF NOT EXISTS terms(
      termid INTEGER PRIMARY KEY AUTOINCREMENT,
      firstterm TEXT,
      secondterm TEXT,
      definition TEXT,
      document TEXT,
      UNIQUE (firstterm, secondterm, definition, document)); 
   """)
   conn.commit() # Если мы не просто читаем, но и вносим изменения в базу данных - необходимо сохранить транзакцию
   return cur, conn

cur, conn = create_db()


In [ ]:
stops = set(stopwords.words('russian'))
stops.add('.')
stops.add(',')
stops.add('Другими')
stops.add('словами')



def term_from_sent(sent, sep = '— это '):
    sent = sent.replace('.', '')
    left_side_sent = sent.split(sep)[0]
    right_side_sent = sent.split(sep)[1]    
    
    term = (left_side_sent, right_side_sent) \
            [len(word_tokenize(left_side_sent.replace('.', '').replace(',', ''))) > \
             len(word_tokenize(right_side_sent.replace('.', '').replace(',', ''))) \
            ] #выделяет наименьшую по количеству слов часть предложения, которая разбивается '— это'
    
    return term.strip()


def concat(terms):      
    terms = [merging_broken_splitted_words(word_tokenize(term)) for term in terms]
    
    if None in terms:
        return None
    
    filtered_words = [' '.join([word for word in term if word not in stops]) for term in terms]
    return filtered_words
        

if __name__ == "__main__":
    extracted_terms = 0
    new_paths = [path for path in os.listdir() if 'simple' in path]
    for path in new_paths:
        soup = read_soup(path)
        
        sents_by_strip = text_from_soup(soup)
        
        sents_by_strip_2 = [sent + '.' for sent in sents_by_strip.split('.')]
        for sent in sents_by_strip_2:
            if ('— это ' in sent) and ('l' not in sent) and ('Например' not in sent):
                
                print(sent)
                terms = [] # список терминов, извлеченных из определения
                
                
                if '(' in sent and ')' in sent:
                    sents_in_brackets = [match[1] for match in re.finditer(r"\(([^()]+)\)", sent)]
                    for sub_sent in sents_in_brackets: # так как может быть несколько пар скобок ()
                        if '— это ' in sub_sent:
                            terms.append(term_from_sent(sub_sent))
                    sent = re.sub(r"\(([^()]+)\)", "", sent)
                
                terms.append(term_from_sent(sent))
                if len(terms) > 1:
                    #print(sent)
                    #print(terms)
                    extracted_terms += 2
                    pass
                else:
                    extracted_terms += 1
                
                print(terms)
                finally_terms = concat(terms)
                finally_terms = [finally_terms_.capitalize() for finally_terms_ in finally_terms]
                print('Термины: ', finally_terms)

                termin = (None, finally_terms[0], (finally_terms[-1], None) [len(finally_terms) == 1],
                sent, 'YaKlass.ru')
                print(termin)
                print()
                cur.execute("INSERT INTO terms VALUES(?, ?, ?, ?, ?);", termin)
                conn.commit()

    print(extracted_terms)

In [ ]:
cur.close()
conn.close()

In [2]:
if __name__ == "__main__":
    all_sents_tokenized = []
    all_sents_tokenized_by_lessons = []
    all_text = []
    definition_sents = []
    definition_raw_sents = [] # 69 штук блоков терминов. тип данных 'list' of 'bs4.BeautifulSoup'
    count = 0
    count_def_texts = 0
    sents_len = 0
    new_paths = [path for path in os.listdir() if 'simple' in path]
    for path in new_paths:
        soup = read_soup(path)
        definition_tags = soup.find_all(class_ = 'gxst-definition')
        count_def_texts += len(definition_tags)
        text = text_from_soup(soup)
        all_text += [text]
        sents = sent_tokenize(text)
        all_sents_tokenized += sents
        all_sents_tokenized_by_lessons += [sents]
        for definition_tag in definition_tags:
            definition_tag_text = text_from_soup(BeautifulSoup(str(definition_tag), "html.parser"))
            if '1' not in definition_tag_text:
                definition_raw_sents += [BeautifulSoup(str(definition_tag), "html.parser")]
                definition_sents += [definition_tag_text]
            if ('l' not in definition_tag_text) and ('m' not in definition_tag_text) and ('1' not in definition_tag_text):
                count += 1
                if definition_tag_text.count('.') > 1:
                    print(definition_tag_text) 
    print(count)
    print(count_def_texts)
    print(sents_len)

 Ломаной называется фигура, которая состоит из точек и соединяющих их отрезков. Точки называются вершинами ломаной, а отрезки — звеньями ломаной. 
 Многоугольник — это простая замкнутая ломаная линия и конечная часть плоскости, которую она ограничивает. Вершины ломаной линии называются вершинами многоугольника, а её звенья — сторонами многоугольника. Отрезок, соединяющий две вершины, не лежащие на одной стороне, называется диагональю многоугольника. 
 Угол — геометрическая фигура, которая состоит из точки и двух лучей, исходящих из этой точки. Эти лучи называют сторонами угла , а их общее начало — вершиной угла . 
 Две плоскости называются параллельными , если они не имеют общих точек. Две прямые в пространстве называются параллельными , если они лежат в одной плоскости и не пересекаются. Прямую называют перпендикулярной к плоскости , если она перпендикулярна к любой прямой в этой плоскости. 
 Окружность — геометрическая фигура, состоящая из всех точек плоскости, которые находятся на з

In [3]:
def common_pos(word):
    return morph.parse(word)[0].tag.POS

def common_case(word):
    return morph.parse(word)[0].tag.case

In [4]:
tokenized_definition_sents = []
for sent in definition_sents:
    print(sent_tokenize(sent))
    tokenized_definition_sents += sent_tokenize(sent)

print(len(tokenized_definition_sents))
print('\n'.join(tokenized_definition_sents))

[' В этом разложении коэффициенты координатных векторов называют координатами вектора m .']
[' Координатами вектора являются координаты конечной точки этого вектора, если вектор расположен так, что его начало находится в начале координат.']
[' Четырёхугольник, все вершины которого лежат на окружности, называется вписанным в эту окружность, а окружность называется описанной около четырёхугольника.']
[' Если все стороны четырёхугольника касаются окружности, то он называется четырёхугольником, описанным около этой окружности, а окружность — вписанной в четырёхугольник.']
[' Окружность называют описанной около треугольника , если все вершины треугольника расположены на окружности.']
[' Окружность называют вписанной в треугольник , если все стороны треугольника касаются окружности.']
[' Измерение — это сравнение объекта измерения с выбранной единицей измерения.']
[' Если прямая имеет две общие точки с окружностью, то она называется секущей .']
[' Касательной к окружности называется прямая, 

In [5]:
tokenized_definition_sents_no_this_sents = [sent for sent in tokenized_definition_sents if '— это ' not in sent]
for t in tokenized_definition_sents_no_this_sents:
    print(t)

print(len(tokenized_definition_sents), len(tokenized_definition_sents_no_this_sents))

 В этом разложении коэффициенты координатных векторов называют координатами вектора m .
 Координатами вектора являются координаты конечной точки этого вектора, если вектор расположен так, что его начало находится в начале координат.
 Четырёхугольник, все вершины которого лежат на окружности, называется вписанным в эту окружность, а окружность называется описанной около четырёхугольника.
 Если все стороны четырёхугольника касаются окружности, то он называется четырёхугольником, описанным около этой окружности, а окружность — вписанной в четырёхугольник.
 Окружность называют описанной около треугольника , если все вершины треугольника расположены на окружности.
 Окружность называют вписанной в треугольник , если все стороны треугольника касаются окружности.
 Если прямая имеет две общие точки с окружностью, то она называется секущей .
 Касательной к окружности называется прямая, имеющая с окружностью одну общую точку.
 Ломаной называется фигура, которая состоит из точек и соединяющих их о

In [6]:
import ast 
terms_filename = '../Предложения, которые были выделены в определения на сайте YaKlass (Извлеченные вручную термины).txt'

with open(terms_filename, encoding = 'utf-8') as file:
    terms_list = [ast.literal_eval(line.rstrip()) for line in file]


In [7]:
# Путь обработки определений
# definition_sents
# tokenized_definition_sents
# tokenized_definition_sents_no_this_sents
# word_case_list

terms_list_tokenized = []
for terms in terms_list:
    for terms_ in terms:
        if type(terms_) is str:
            terms_list_tokenized += [terms]
            break
        if type(terms_) is list:
            terms_list_tokenized += [terms_]

this_mask = [int(not('— это ' in sent)) for sent in (tokenized_definition_sents)]

print(this_mask)

tokenized_definition_sents_no_this_sents = [sent for ind, sent in enumerate(tokenized_definition_sents) if this_mask[ind] == 1]
terms_list_this_mask = [terms for ind, terms in enumerate(terms_list_tokenized) if this_mask[ind] == 1]
print(terms_list_this_mask)

word_case_list = []
for sent in tokenized_definition_sents_no_this_sents:
    word_case_list += [[(word, common_case(word)) for word in word_tokenize(sent.strip())]]

word_case_list_no_none = [[(word, case) for word, case in word_case_tuples if case != None] for word_case_tuples in word_case_list]

[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[['Координаты вектора'], ['Координаты вектора'], ['Четырёхугольник, вписанный в окружность', 'Окружность, описанная около четырёхугольника'], ['Четырёхугольник, описанный около окружности', 'Окружность, вписанная в четырёхугольник'], ['Описанная окружность треугольника'], ['Вписанная окружность треугольника'], ['Секущая прямая'], ['Касательная к окружности'], ['Ломаная'], ['Вершина ломаной', 'Звенья ломаной'], ['Вершина многоугольника', 'Сторона многоугольника'], ['Диагональ многоугольника'], ['Луч', 'Начало луча'], ['Угол'], ['Сторона угла', 'Вершина угла'], ['Развёрнутый угол'], ['Перпендикуляр'], ['Равнобедренный треугольник'], ['Равносторонний треугольник'], ['Многогранник'], ['Параллельные плоскости'], ['Параллельные прямые в пространстве'], ['

In [19]:
import xlsxwriter
def make_excel_file(word_case_list, name, workbook, indent = 0):
    worksheet = workbook.add_worksheet(name)
    
    for IND, word_case_tuples in enumerate(word_case_list):
        print(word_case_tuples)
        for ind, (word, case) in enumerate(word_case_tuples):
            worksheet.write(3*IND, indent + ind, word)
            worksheet.write(3*IND + 1, indent + ind, str(case))
    return worksheet

workbook = xlsxwriter.Workbook('Предложения и падежи.xlsx')
worksheet_1 = make_excel_file(word_case_list, 'Предложения и падежи', workbook, indent = 2)

for IND, terms_ in enumerate(terms_list_this_mask):
    worksheet_1.write(3 * IND, 0, str(terms_))

worksheet_2 = make_excel_file(word_case_list_no_none, 'Предложения и падежи без None', workbook)

for IND, terms_ in enumerate(terms_list_this_mask):
    worksheet_2.write(3 * IND, 20, str(terms_))
workbook.close()


[('В', None), ('этом', 'loct'), ('разложении', 'loct'), ('коэффициенты', 'nomn'), ('координатных', 'gent'), ('векторов', 'gent'), ('называют', None), ('координатами', 'ablt'), ('вектора', 'gent'), ('m', None), ('.', None)]
[('Координатами', 'ablt'), ('вектора', 'gent'), ('являются', None), ('координаты', 'nomn'), ('конечной', 'gent'), ('точки', 'gent'), ('этого', 'gent'), ('вектора', 'gent'), (',', None), ('если', None), ('вектор', 'nomn'), ('расположен', None), ('так', None), (',', None), ('что', None), ('его', 'accs'), ('начало', 'nomn'), ('находится', None), ('в', None), ('начале', 'loct'), ('координат', 'gent'), ('.', None)]
[('Четырёхугольник', 'nomn'), (',', None), ('все', None), ('вершины', 'gent'), ('которого', 'gent'), ('лежат', None), ('на', None), ('окружности', 'gent'), (',', None), ('называется', None), ('вписанным', 'ablt'), ('в', None), ('эту', 'accs'), ('окружность', 'nomn'), (',', None), ('а', None), ('окружность', 'nomn'), ('называется', None), ('описанной', 'gent'), 

In [8]:
word_case_pos_list = []
for sent in tokenized_definition_sents_no_this_sents:
    word_case_pos_list += [[(word, common_case(word), common_pos(word)) for word in word_tokenize(sent.strip())]]

word_case_pos_list_no_none = [[(word, case, pos) for word, case, pos in word_case_pos_tuples if case != None] for word_case_pos_tuples in word_case_pos_list]

In [21]:
def make_excel_file_big(word_case_pos_list, name, workbook, indent = 0):
    worksheet = workbook.add_worksheet(name)
    
    for IND, word_case_pos_tuples in enumerate(word_case_pos_list):
        print(word_case_pos_tuples)
        for ind, (word, case, pos) in enumerate(word_case_pos_tuples):
            worksheet.write(4*IND, indent + ind, word)
            worksheet.write(4*IND + 1, indent + ind, str(case))
            worksheet.write(4*IND + 2, indent + ind, str(pos))
    return worksheet

workbook = xlsxwriter.Workbook('Предложения и падежи BIG.xlsx')
worksheet_1 = make_excel_file_big(word_case_pos_list, 'Sents, cases, pos', workbook, indent = 2)

for IND, terms_ in enumerate(terms_list_this_mask):
    worksheet_1.write(4 * IND, 0, str(terms_))

worksheet_2 = make_excel_file_big(word_case_pos_list_no_none, 'Sents, cases, pos без None', workbook)

for IND, terms_ in enumerate(terms_list_this_mask):
    worksheet_2.write(4 * IND, 20, str(terms_))
workbook.close()


[('В', None, 'PREP'), ('этом', 'loct', 'NPRO'), ('разложении', 'loct', 'NOUN'), ('коэффициенты', 'nomn', 'NOUN'), ('координатных', 'gent', 'ADJF'), ('векторов', 'gent', 'NOUN'), ('называют', None, 'VERB'), ('координатами', 'ablt', 'NOUN'), ('вектора', 'gent', 'NOUN'), ('m', None, None), ('.', None, None)]
[('Координатами', 'ablt', 'NOUN'), ('вектора', 'gent', 'NOUN'), ('являются', None, 'VERB'), ('координаты', 'nomn', 'NOUN'), ('конечной', 'gent', 'ADJF'), ('точки', 'gent', 'NOUN'), ('этого', 'gent', 'NPRO'), ('вектора', 'gent', 'NOUN'), (',', None, None), ('если', None, 'CONJ'), ('вектор', 'nomn', 'NOUN'), ('расположен', None, 'PRTS'), ('так', None, 'CONJ'), (',', None, None), ('что', None, 'CONJ'), ('его', 'accs', 'NPRO'), ('начало', 'nomn', 'NOUN'), ('находится', None, 'VERB'), ('в', None, 'PREP'), ('начале', 'loct', 'NOUN'), ('координат', 'gent', 'NOUN'), ('.', None, None)]
[('Четырёхугольник', 'nomn', 'NOUN'), (',', None, None), ('все', None, 'PRCL'), ('вершины', 'gent', 'NOUN'), 

In [9]:
for ind, sent in enumerate(tokenized_definition_sents_no_this_sents): # 7 определений с разным кол-вом терминов реально существующих и по версии term_keyword_count()
    if term_keyword_count(sent) != len(terms_list_this_mask[ind]):
        print(term_keyword_count(sent), len(terms_list_this_mask[ind]))
        print(sent, terms_list_this_mask[ind])


2 1
 Если обе стороны угла являются дополнительными полупрямыми, угол называют развёрнутым . ['Развёрнутый угол']
2 1
 Четырёхугольная призма, основания которой — параллелограммы, называется параллелепипедом . ['Параллелепипед']
0 1
Это преобразование, в котором получаются подобные фигуры (фигуры, у которых соответствующие углы равны и стороны пропорциональны). ['Подобные фигуры']
2 1
 Два угла, у которых одна сторона общая, а две другие являются продолжением одна другой, называются с межными . ['Смежные углы']
2 1
 Два угла называются вертикальными , если обе стороны одного угла являются продолжениями сторон другого. ['Вертикальные углы']
1 2
 Векторными величинами, или векторами, называют величины, имеющие и численное значение, и направление. ['Векторные величины', 'Вектор']
0 2
Проекция начала вектора соответствует началу геометрической проекции, а проекция конца вектора соответствует концу геометрической проекции. ['Начало геометрической проекции', 'Конец геометрической проекции']


In [10]:
cur, conn = create_db()

In [39]:
# GUI приложения "Система для извлечения терминов"
import tkinter as tk

class TermExcractor(tk.Tk):
    def __init__(self):
        super().__init__()

        self.definition_index = 0
        self.definitions = tokenized_definition_sents_no_this_sents[self.definition_index]

        self.up_sents = 0
        self.down_sents = 0


        self.definition_frame = tk.LabelFrame(self, text = 'Определение')
        self.definition_label = tk.Message(self.definition_frame, text = self.definitions, width = 500)
        self.definition_label.pack()
        tk.Button(self.definition_frame, text = 'Вверх', command = self.up).pack()
        tk.Button(self.definition_frame, text = 'Вниз', command = self.down).pack()
        self.definition_frame.pack(side = tk.TOP)

        self.first_term_frame = tk.LabelFrame(self, text = 'Первый термин')
        self.first_definition_term = tk.Text(self.first_term_frame, width = 100, height = 3)
        self.first_definition_term.pack()
        self.first_term_frame.pack(side = tk.TOP)

        self.second_term_frame = tk.LabelFrame(self, text = 'Второй термин (если нет, то оставить пустым)')
        self.second_definition_term = tk.Text(self.second_term_frame, width = 100, height = 3)
        self.second_definition_term.pack()
        self.second_term_frame.pack(side = tk.TOP)


        self.arrow_buttons_frame = tk.Frame(self)
        tk.Button(self.arrow_buttons_frame, text = '<--', width = 5, height = 2, command = self.prev).pack(side = tk.LEFT)
        tk.Button(self.arrow_buttons_frame, text = '-->', width = 5, height = 2, command = self.next).pack(side = tk.RIGHT)
        self.arrow_buttons_frame.pack()

        self.save_frame = tk.Frame(self)
        self.save_btn = tk.Button(self.save_frame, text = 'Сохранить термин', width = 15, height = 2, command = self.save_term)
        self.save_btn.pack(side = tk.RIGHT)
        self.save_frame.pack()

    def up(self):
        self.up_sents += 1
        if self.definition_index - self.up_sents >= 0:
            self.definitions =  tokenized_definition_sents_no_this_sents[self.definition_index - self.up_sents] + self.definitions
            self.definition_label.config(text = self.definitions)
    
    def down(self):
        self.down_sents += 1
        if self.definition_index + self.up_sents < len(tokenized_definition_sents):
            self.definitions = self.definitions + tokenized_definition_sents_no_this_sents[self.definition_index + self.down_sents]
            self.definition_label.config(text = self.definitions)

    def prev(self):
        if self.definition_index == 0:
            return
        self.definition_index -= 1
        self.definitions = tokenized_definition_sents_no_this_sents[self.definition_index]
        self.definition_label.config(text = self.definitions)

        self.first_definition_term.delete("1.0", tk.END)
        self.second_definition_term.delete("1.0", tk.END)

        self.up_sents = 0
        self.down_sents = 0

        self.save_btn.config(background = 'white')

    def next(self):
        if self.definition_index == len(tokenized_definition_sents_no_this_sents) - 1:
            return
        self.definition_index += 1
        self.definitions = tokenized_definition_sents_no_this_sents[self.definition_index]
        self.definition_label.config(text = self.definitions)

        self.first_definition_term.delete("1.0", tk.END)
        self.second_definition_term.delete("1.0", tk.END)

        self.up_sents = 0
        self.down_sents = 0

        self.save_btn.config(background = 'white')

    def save_term(self):
        first_term = self.first_definition_term.get('1.0', tk.END)[:-1]
        second_term = self.second_definition_term.get('1.0', tk.END)[:-1]
        if (first_term == '') and (second_term == ''):
            return
        
        termin = (None, first_term, (second_term, None) [second_term == ''],
        self.definitions, 'YaKlass.ru')

        cur.execute("INSERT INTO terms VALUES(?, ?, ?, ?, ?);", termin) # : Таким способом получается избежать SQL-инъекций
        conn.commit()

        self.save_btn.config(background = 'green')

application = TermExcractor()
application.title('Система для извлечения терминов')
application.geometry('880x650')
application.mainloop()



string


In [12]:
tokenized_definition_sents_no_this_sents

[' В этом разложении коэффициенты координатных векторов называют координатами вектора m .',
 ' Координатами вектора являются координаты конечной точки этого вектора, если вектор расположен так, что его начало находится в начале координат.',
 ' Четырёхугольник, все вершины которого лежат на окружности, называется вписанным в эту окружность, а окружность называется описанной около четырёхугольника.',
 ' Если все стороны четырёхугольника касаются окружности, то он называется четырёхугольником, описанным около этой окружности, а окружность — вписанной в четырёхугольник.',
 ' Окружность называют описанной около треугольника , если все вершины треугольника расположены на окружности.',
 ' Окружность называют вписанной в треугольник , если все стороны треугольника касаются окружности.',
 ' Если прямая имеет две общие точки с окружностью, то она называется секущей .',
 ' Касательной к окружности называется прямая, имеющая с окружностью одну общую точку.',
 ' Ломаной называется фигура, которая с

In [26]:
terms_filename = '../Дипломная работа последнее/DIPLOMNAYA/ЯКласс/YaKlass'

with open(terms_filename, encoding = 'utf-8') as file:
    terms_list = [ast.literal_eval(line.rstrip()) for line in file]


FileNotFoundError: [Errno 2] No such file or directory: '../Дипломная работа последнее/DIPLOMNAYA/ЯКласс/YaKlass'

In [4]:
import os
os.chdir('../Дипломная работа последнее/DIPLOMNAYA/ЯКласс/YaKlass')

In [33]:
another_sents = []
for lesson_sents in all_sents_tokenized_by_lessons:
    for sent in lesson_sents:
        if (term_keyword_count(sent) >= 1) and ('l' not in sent) and ('m' not in sent):
            another_sents += [sent]

another_sents = [sent for sent in another_sents if '— это' not in sent]
print('\n'.join(another_sents))

 Четырёхугольник, все вершины которого лежат на окружности, называется вписанным в эту окружность, а окружность называется описанной около четырёхугольника.
Все углы четырёхугольника являются вписанными в окружность, значит, равны половине дуг, на которые опираются.
Самостоятельно сделай обзор четырёхугольников (параллелограмм, в том числе — квадрат, прямоугольник, ромб, трапеция, в том числе — равнобедренная трапеция и прямоугольная трапеция), около которых можно описать окружность.
 Если все стороны четырёхугольника касаются окружности, то он называется четырёхугольником, описанным около этой окружности, а окружность — вписанной в четырёхугольник.
Самостоятельно сделай обзор четырёхугольников (параллелограмм, в том числе — квадрат, прямоугольник, ромб, трапеция, в том числе — равнобедренная трапеция и прямоугольная трапеция), в которые можно вписать окружность.
Окружность называют описанной около треугольника , если все вершины треугольника расположены на окружности.
Окружность назыв

In [22]:
tokenized_definition_sents_no_this_sents

[' В этом разложении коэффициенты координатных векторов называют координатами вектора m .',
 ' Координатами вектора являются координаты конечной точки этого вектора, если вектор расположен так, что его начало находится в начале координат.',
 ' Четырёхугольник, все вершины которого лежат на окружности, называется вписанным в эту окружность, а окружность называется описанной около четырёхугольника.',
 ' Если все стороны четырёхугольника касаются окружности, то он называется четырёхугольником, описанным около этой окружности, а окружность — вписанной в четырёхугольник.',
 ' Окружность называют описанной около треугольника , если все вершины треугольника расположены на окружности.',
 ' Окружность называют вписанной в треугольник , если все стороны треугольника касаются окружности.',
 ' Если прямая имеет две общие точки с окружностью, то она называется секущей .',
 ' Касательной к окружности называется прямая, имеющая с окружностью одну общую точку.',
 ' Ломаной называется фигура, которая с

In [39]:
another_sents_2 = [sent for sent in all_sents_tokenized if (term_keyword_count(sent) >= 1) and ('l' not in sent) and ('m' not in sent)]
print('\n'.join(another_sents_2))

 Четырёхугольник, все вершины которого лежат на окружности, называется вписанным в эту окружность, а окружность называется описанной около четырёхугольника.
Все углы четырёхугольника являются вписанными в окружность, значит, равны половине дуг, на которые опираются.
Самостоятельно сделай обзор четырёхугольников (параллелограмм, в том числе — квадрат, прямоугольник, ромб, трапеция, в том числе — равнобедренная трапеция и прямоугольная трапеция), около которых можно описать окружность.
 Если все стороны четырёхугольника касаются окружности, то он называется четырёхугольником, описанным около этой окружности, а окружность — вписанной в четырёхугольник.
Самостоятельно сделай обзор четырёхугольников (параллелограмм, в том числе — квадрат, прямоугольник, ромб, трапеция, в том числе — равнобедренная трапеция и прямоугольная трапеция), в которые можно вписать окружность.
Окружность называют описанной около треугольника , если все вершины треугольника расположены на окружности.
Окружность назыв

In [74]:
all_text_with_keyword = []
for text_ in all_text:
    sents_by_strip_2 = [sent + '.' for sent in text_.split('.')]
    for sent in sents_by_strip_2:
        if (term_keyword_count(sent) >= 1) and ('l' not in sent) and ('m' not in sent):
            all_text_with_keyword += [sent]

all_text_with_keyword = [sent.strip() for sent in all_text_with_keyword if '— это' not in sent]


tokenized_definition_sents_no_this_sents_no_math = [sent.strip() for sent in tokenized_definition_sents_no_this_sents if ('l' not in sent) and ('m' not in sent)]
# for sent in tokenized_definition_sents_no_this_sents_no_math:
#     if sent not in all_text_with_keyword:
        # print(sent)

all_text_with_keyword = [sent for sent in all_text_with_keyword if sent not in tokenized_definition_sents_no_this_sents_no_math]
print('\n'.join(all_text_with_keyword))

Все углы четырёхугольника являются вписанными в окружность, значит, равны половине дуг, на которые опираются.
Самостоятельно сделай обзор четырёхугольников (параллелограмм, в том числе — квадрат, прямоугольник, ромб, трапеция, в том числе — равнобедренная трапеция и прямоугольная трапеция), около которых можно описать окружность.
Самостоятельно сделай обзор четырёхугольников (параллелограмм, в том числе — квадрат, прямоугольник, ромб, трапеция, в том числе — равнобедренная трапеция и прямоугольная трапеция), в которые можно вписать окружность.
Равносторонний треугольник Обрати внимание! У равностороннего треугольника совпадают биссектрисы, медианы и высоты, то есть, эти отрезки являются также серединными перпендикулярами.
Третий признак позволяет назвать треугольник очень сильной, устойчивой фигурой, иногда говорят, что треугольник — жёсткая фигура .
Такие числа называются иррациональными.
Первая замечательная точка треугольника — точка пересечения биссектрис.
Вторая замечательная точк

In [75]:
# Tkinter по добавлению 156 (155) дефиниционных предложений
import tkinter as tk

class TermExcractor(tk.Tk):
    def __init__(self):
        super().__init__()

        self.definition_index = 0
        self.definitions = all_text_with_keyword[self.definition_index]

        self.up_sents = 0
        self.down_sents = 0

        self.definition_frame = tk.LabelFrame(self, text = 'Определение')
        self.definition_label = tk.Message(self.definition_frame, text = self.definitions, width = 500)
        self.definition_label.pack()
        tk.Button(self.definition_frame, text = 'Вверх', command = self.up).pack()
        tk.Button(self.definition_frame, text = 'Вниз', command = self.down).pack()
        self.definition_frame.pack(side = tk.TOP)

        self.first_term_frame = tk.LabelFrame(self, text = 'Первый термин')
        self.first_definition_term = tk.Text(self.first_term_frame, width = 100, height = 3)
        self.first_definition_term.pack()
        self.first_term_frame.pack(side = tk.TOP)

        self.second_term_frame = tk.LabelFrame(self, text = 'Второй термин (если нет, то оставить пустым)')
        self.second_definition_term = tk.Text(self.second_term_frame, width = 100, height = 3)
        self.second_definition_term.pack()
        self.second_term_frame.pack(side = tk.TOP)


        self.arrow_buttons_frame = tk.Frame(self)
        tk.Button(self.arrow_buttons_frame, text = '<--', width = 5, height = 2, command = self.prev).pack(side = tk.LEFT)
        tk.Button(self.arrow_buttons_frame, text = '-->', width = 5, height = 2, command = self.next).pack(side = tk.RIGHT)
        self.arrow_buttons_frame.pack()

        self.save_frame = tk.Frame(self)
        self.save_btn = tk.Button(self.save_frame, text = 'Сохранить термин', width = 15, height = 2, command = self.save_term)
        self.save_btn.pack(side = tk.RIGHT)
        self.save_frame.pack()

    def up(self):
        self.up_sents += 1
        if self.definition_index - self.up_sents >= 0:
            self.definitions =  tokenized_definition_sents_no_this_sents[self.definition_index - self.up_sents] + self.definitions
            self.definition_label.config(text = self.definitions)
    
    def down(self):
        self.down_sents += 1
        if self.definition_index + self.up_sents < len(tokenized_definition_sents):
            self.definitions = self.definitions + tokenized_definition_sents_no_this_sents[self.definition_index + self.down_sents]
            self.definition_label.config(text = self.definitions)

    def prev(self):
        if self.definition_index == 0:
            return
        self.definition_index -= 1
        self.definitions = all_text_with_keyword[self.definition_index]
        self.definition_label.config(text = self.definitions)

        self.first_definition_term.delete("1.0", tk.END)
        self.second_definition_term.delete("1.0", tk.END)

        self.up_sents = 0
        self.down_sents = 0

        self.save_btn.config(background = 'white')

    def next(self):
        if self.definition_index == len(all_text_with_keyword) - 1:
            return
        self.definition_index += 1
        self.definitions = all_text_with_keyword[self.definition_index]
        self.definition_label.config(text = self.definitions)

        self.first_definition_term.delete("1.0", tk.END)
        self.second_definition_term.delete("1.0", tk.END)

        self.up_sents = 0
        self.down_sents = 0

        self.save_btn.config(background = 'white')

    def save_term(self):
        first_term = self.first_definition_term.get('1.0', tk.END)[:-1]
        second_term = self.second_definition_term.get('1.0', tk.END)[:-1]
        if (first_term == '') and (second_term == ''):
            return
        
        termin = (None, first_term, (second_term, None) [second_term == ''],
        self.definitions, 'YaKlass.ru')

        cur.execute("INSERT INTO terms VALUES(?, ?, ?, ?, ?);", termin) # : Таким способом получается избежать SQL-инъекций
        conn.commit()

        self.save_btn.config(background = 'green')

application = TermExcractor()
application.title('Система для извлечения терминов')
application.geometry('880x650')
application.mainloop()

In [ ]:
cur.close()
conn.close()